In [ ]:
from CLIP import CLIP

In [2]:
from encoder import TextEncoder, ImageEncoder
from projection_head import ProjectionHead

text_encoder = TextEncoder()
image_encoder = ImageEncoder()

Loading Text Encoder....
Loading Image Encoder....


In [3]:
t_projection = ProjectionHead(512)
i_projection = ProjectionHead(768)

In [4]:
clip = CLIP(text_encoder, image_encoder, t_projection, i_projection)

In [6]:
# from PIL import Image
# import requests
# 
# url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
# image = Image.open(requests.get(url, stream=True).raw)
# text = "Two dogs sleeping"

In [7]:
# out = clip(image, text)
# out

In [5]:
# Read data from CSV with 2 columns image, captions and each image has 5 captions, I want to use only one caption
# Return different arrays for images and texts and make sure the order is the same

import pandas as pd
import numpy as np

# Read the CSV file
data = pd.read_csv('data/captions.csv', names=['image', 'captions'], header=None)

# Group by 'image' and select the first 'caption' for each group
grouped_data = data.groupby('image').first().reset_index()

# Append 'data/Images' to each image name
grouped_data['image'] = grouped_data['image'].apply(lambda x: 'data/Images/' + x)

# Convert the 'image' and 'caption' columns to numpy arrays
images = grouped_data['image'].to_numpy()
captions = grouped_data['captions'].to_numpy()


In [66]:
images = images[:50]
captions = captions[:50]

In [67]:
from train import CLIPDataset
from torch.utils.data import DataLoader
from torchvision import transforms
# 
# # Create sample array of texts and images
# texts = ["A photo of a dog", "A photo of a bird"]
# images = ["images/dog.jpeg", "images/bird.jpg"]

# Create transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
# Create dataset and dataloader
dataset = CLIPDataset(captions, images, transform)
dataloader = DataLoader(dataset, batch_size=16)

In [68]:
# for x in dataloader:
#     print(x)
#     break

In [69]:
from torch.optim import Adam
from train import train

optimizer = Adam(clip.parameters(), lr=1e-4)

train(clip, dataloader, optimizer, 40)

Epoch 0, Batch 0, Loss: 2.9951376914978027
Epoch 0, Batch 1, Loss: 3.0591814517974854
Epoch 0, Batch 2, Loss: 3.2445356051127114
Epoch 0, Batch 3, Loss: 3.0767226219177246
Epoch 0, Loss: 3.0767226219177246
Epoch 1, Batch 0, Loss: 3.141519546508789
Epoch 1, Batch 1, Loss: 3.2237559159596763
Epoch 1, Batch 2, Loss: 3.3347324643816267
Epoch 1, Batch 3, Loss: 3.6751158833503723
Epoch 1, Loss: 3.6751158833503723
Epoch 2, Batch 0, Loss: 3.701079845428467
Epoch 2, Batch 1, Loss: 3.676063656806946
Epoch 2, Batch 2, Loss: 3.6709263541481714
Epoch 2, Batch 3, Loss: 3.4768966337045035
Epoch 2, Loss: 3.4768966337045035
Epoch 3, Batch 0, Loss: 3.4559492606383104
Epoch 3, Batch 1, Loss: 3.4614257557051524
Epoch 3, Batch 2, Loss: 3.4925103267033895
Epoch 3, Batch 3, Loss: 3.4777611270546913
Epoch 3, Loss: 3.4777611270546913
Epoch 4, Batch 0, Loss: 3.4740990260068108
Epoch 4, Batch 1, Loss: 3.4700889918539257
Epoch 4, Batch 2, Loss: 3.4746300609488237
Epoch 4, Batch 3, Loss: 3.3682191610336303
Epoch 4

In [70]:
# Set the model to evaluation mode
clip.eval()

CLIP(
  (text_encoder): TextEncoder(
    (model): T5EncoderModel(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(in_features=512, out_features=512, bias=False)
                  (k): Linear(in_features=512, out_features=512, bias=False)
                  (v): Linear(in_features=512, out_features=512, bias=False)
                  (o): Linear(in_features=512, out_features=512, bias=False)
                  (relative_attention_bias): Embedding(32, 8)
                )
                (layer_norm): T5LayerNorm()
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (1): T5LayerFF(
                (DenseReluDense): T5DenseActDense(
                  (wi): Linear(in_features=512, out_features=2048

In [73]:
# Test the model
from PIL import Image
import requests

image = Image.open("data/Images/1000268201_693b08cb0e.jpg")
text = "A child in a pink dress is climbing up a set of stairs in an entry way ."

_, _, text_features, image_features = clip(image, text)

# Import softmax function
from torch.nn.functional import softmax

# Calculate the similarity score
similarity = text_features @ image_features.T
similarity


tensor([[12.4456]], grad_fn=<MmBackward0>)

In [74]:
# Select 10 examples for testing
test_images = images[:4]
test_captions = captions[:4]
print(test_images)
print(test_captions)

['data/Images/1000268201_693b08cb0e.jpg'
 'data/Images/1001773457_577c3a7d70.jpg'
 'data/Images/1002674143_1b742ab4b8.jpg'
 'data/Images/1003163366_44323f5815.jpg']
['A child in a pink dress is climbing up a set of stairs in an entry way .'
 'A black dog and a spotted dog are fighting'
 'A little girl covered in paint sits in front of a painted rainbow with her hands in a bowl .'
 'A man lays on a bench while his dog sits by him .']


In [78]:
# Test the model
from PIL import Image
import requests

image = Image.open("data/Images/1001773457_577c3a7d70.jpg")
text = "A black dog and a spotted dog are fighting"

_, _, text_features, image_features = clip(image, text)

# Import softmax function
from torch.nn.functional import softmax

# Calculate the similarity score
similarity = text_features @ image_features.T
similarity


tensor([[12.9002]], grad_fn=<MmBackward0>)

In [79]:
import torch
test_image = ['data/Images/1000268201_693b08cb0e.jpg',
              'data/Images/1001773457_577c3a7d70.jpg',
              'data/Images/1002674143_1b742ab4b8.jpg',
              'data/Images/1003163366_44323f5815.jpg']
test_text = ['A child in a pink dress is climbing up a set of stairs in an entry way .',
             'A black dog and a spotted dog are fighting',
             'A little girl covered in paint sits in front of a painted rainbow with her hands in a bowl .',
             'A man lays on a bench while his dog sits by him .']



# Pass test data through model
t_im = Image.open('data/Images/1002674143_1b742ab4b8.jpg')
test_image = [Image.open(image) for image in test_image]
_, _, test_text_features, test_image_features = clip(test_image[0], test_text[0])

# Compute similarity scores
for i in range(1, len(test_image)):
    _, _, text_features, image_features = clip(test_image[i], test_text[i])
    test_text_features = torch.cat((test_text_features, text_features), dim=0)
    test_image_features = torch.cat((test_image_features, image_features), dim=0)

In [49]:
text_features, image_features

(tensor([[ 3.4426e-01, -4.8625e-01,  3.6989e-01,  1.9431e-01, -8.1173e-02,
           1.5413e-01,  5.5961e-01, -6.9249e-01, -9.1896e-01,  2.4130e-01,
           1.7374e-02, -9.3727e-02,  2.5287e+00, -1.3002e+00,  5.4595e-01,
          -2.7917e-01, -1.4924e+00, -7.3973e-01, -9.0295e-01, -1.3562e-01,
          -7.8348e-01, -7.8147e-01,  2.0823e-01, -4.8749e-01, -9.5894e-01,
           9.2712e-01,  4.2492e-01, -2.3245e-02,  1.5966e-01,  6.9062e-01,
           4.6569e-01,  5.1678e-01,  1.6644e-03,  2.4607e-01,  8.7628e-01,
           3.2009e-01, -3.1724e+00,  3.0375e-02, -4.8416e-01,  1.6206e-01,
           3.1184e-01,  4.8356e-01, -1.0057e+00, -1.6305e+00, -2.9304e-01,
          -2.2038e-01, -1.9418e+00,  5.0844e-01, -1.2323e+00,  1.4476e-01,
           3.4318e-01, -6.2782e-02, -1.1870e+00,  5.0257e-01,  1.7101e+00,
           4.5517e-01, -5.5623e-01,  4.5230e-02, -5.5685e-01, -9.3072e-02,
           6.5650e-01, -9.4037e-01, -8.2675e-01, -7.7307e-02, -1.4902e-01,
          -2.2339e-01,  7

In [85]:
similarity_scores = test_text_features @ test_image_features.T
similarity_scores = similarity_scores.detach().numpy()

In [86]:
from scipy.special import softmax

# Apply softmax along rows
softmax_scores = softmax(similarity_scores, axis=1)

print(softmax_scores)

[[0.08687177 0.4575418  0.10596867 0.34961778]
 [0.09747648 0.47073233 0.11141967 0.32037115]
 [0.07930483 0.47848603 0.0993202  0.34288827]
 [0.06628865 0.47623667 0.09041051 0.36706358]]
